In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 18.7 MB/s eta 0:00:00


In [4]:
import re
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
from google.colab import drive
import pandas as pd
from datetime import datetime
from pandas.io.formats.info import pprint_thing

In [5]:
season_duration = pd.read_excel('/content/drive/MyDrive/DS105L group project/media/season_duration_1.xlsx')
season_duration_range = season_duration['duration'].to_list()
duration = []
for durations in season_duration_range:
  durations = durations.replace('\xa0',' ')
  durations = durations.replace('-', '–')
  start_date_str, end_date_str = durations.split(" – ")
  end_date_str = end_date_str[:-1] + re.sub('[^0-9]','', end_date_str[-1])
  
  # Parse the start and end dates into datetime objects
  start_date = datetime.strptime(start_date_str, "%B %d, %Y")
  end_date = datetime.strptime(end_date_str, "%B %d, %Y")

  # Format the start and end dates as strings with numerical month and year
  start_date_num = start_date.strftime("%Y%m%d")
  end_date_num = end_date.strftime("%Y%m%d")

  duration.append([int(start_date_num), int(end_date_num)])

season_duration['duration'] = duration

display(season_duration)

,season,duration
0,1950-51,"[19501031, 19510318]"
1,1951-52,"[19511101, 19520316]"
2,1952-53,"[19521031, 19530317]"
3,1953-54,"[19531030, 19540314]"
4,1954-55,"[19541030, 19550314]"
...,...,...
68,2018-19,"[20181016, 20190410]"
69,2019-20,"[20191022, 20200311]"
70,2020-21,"[20201222, 20210516]"
71,2021-22,"[20211019, 20220410]"


In [6]:
#connect to google drive
mvp_ladder = pd.read_csv('/content/drive/MyDrive/DS105L group project/media/nba_mvp_ladder_links.csv')
merged_players_Data = pd.read_csv('/content/drive/MyDrive/DS105L group project/merged_data/merged_player_data_2010_2023.csv')
#season_duration = pd.read_excel('/content/drive/MyDrive/DS105L group project/media/season_duration_1.xlsx')
#display(merged_players_Data)
#display(mvp_ladder)
#display(season_duration)

#divide the data by season
news_time_list = mvp_ladder['time'].to_list()
season_duration_range = season_duration['duration'].to_list()

date_list = []
for date in news_time_list:
  date_obj = datetime.strptime(date, '%B %d, %Y')
  numerical_date = date_obj.strftime('%Y%m%d')
  date_list.append(int(numerical_date))

season_transferred = []
for news_date in date_list:
  #print(news_date)
  found_season = False
  for period in duration:
    if period[0] <= news_date <= period[1]:
      row_index = [x[0] for x in season_duration.iloc[:,1]].index(period[0])
      #row_index,column_index = pd.DataFrame(season_duration.iloc[:,1][0] == period[0])
      #print(row_index)
      season = season_duration.iloc[row_index,0]
      #print(season)
      season_transferred.append(season)
      found_season = True
      break
  if found_season == False:
    #print(f"No season found for news date {news_date}")
    season_transferred.append('off-season news')

#print(len(season_transferred))

mvp_ladder['time'] = season_transferred

display(mvp_ladder)

  


,Unnamed: 0,title,time,link
0,0,Kia MVP Ladder: Embiid passes Jokic for No. 1,2022-23,https://www.nba.com/news/kia-mvp-ladder-march-...
1,1,Kia MVP Ladder: Embiid closing in on Jokic,2022-23,https://www.nba.com/news/kia-mvp-ladder-march-...
2,2,Kia MVP Ladder: Tatum rises to No. 3; Jokic st...,2022-23,https://www.nba.com/news/kia-mvp-ladder-march-...
3,3,Kia MVP Ladder: How Top 5 looks at All-Star break,2022-23,https://www.nba.com/news/kia-mvp-ladder-feb-17...
4,4,Kia MVP Ladder: Taking stock after a wild trad...,2022-23,https://www.nba.com/news/kia-mvp-ladder-feb-10...
...,...,...,...,...
1413,1413,Kia Race to the MVP Ladder: Not even Russell W...,2016-17,https://www.nba.com/news/mvp-ladder-week-5-man...
1414,1414,Kia Race to the MVP Ladder: Russell Westbrook ...,2016-17,https://www.nba.com/news/mvp-ladder-week-4-rus...
1415,1415,Kia Race to the MVP Ladder: Anthony Davis craf...,2016-17,https://www.nba.com/news/mvp-ladder-week-3-ant...
1416,1416,Kia Race to the MVP Ladder: How long until a b...,2016-17,https://www.nba.com/news/mvp-ladder-week-2-201...


In [24]:
archive_0_600 = pd.read_csv('/content/drive/MyDrive/DS105L group project/season_date_transferred/nba_news_archive_600.csv')
archive_600_1200 = pd.read_csv('/content/drive/MyDrive/DS105L group project/season_date_transferred/nba_news_archive_600_1200.csv')
archive_1200_1640 = pd.read_csv('/content/drive/MyDrive/DS105L group project/season_date_transferred/nba_news_archive_1200_1640.csv')

def date_season_transfer(dataframe):

  time_list = dataframe['time'].to_list()
  date_list = []

  for date in time_list:
    date_obj = datetime.strptime(date, '%B %d, %Y')
    numerical_date = date_obj.strftime('%Y%m%d')
    date_list.append(int(numerical_date))

  season_duration_list = list(season_duration.iloc[:,1])

  season_transferred = []
  for new_date in date_list:
    found_season = False
    for period in duration:
      if period[0] <= new_date <= period[1]:
        row_index = [x[0] for x in season_duration_list].index(period[0])
        #row_index,column_index = pd.DataFrame(season_duration.iloc[:,1][0] == period[0])
        #print(row_index)
        season = season_duration.iloc[row_index,0]
        #print(season)
        season_transferred.append(season)
        found_season = True
        break
    if found_season == False:
      #print(f"No season found for news date {news_date}")
      season_transferred.append('off-season news')
      
  dataframe['time'] = season_transferred

  return dataframe

drive.mount('/content/drive')

df1 = date_season_transfer(archive_0_600)
df2 = date_season_transfer(archive_600_1200)
df3 = date_season_transfer(archive_1200_1640)

df1.to_csv('/content/drive/MyDrive/DS105L group project/season_date_transferred/with_season_0_600.csv')
df2.to_csv('/content/drive/MyDrive/DS105L group project/season_date_transferred/with_season_600_1200.csv')
df3.to_csv('/content/drive/MyDrive/DS105L group project/season_date_transferred/with_season_1200_1640.csv')

display(df1)
display(df2)
display(df3)
  


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Unnamed: 0,title,time,url
0,0,"Bucks’ George Hill Returns To Milwaukee, Encou...",off-season news,http://global.nba.com/news/bucks-george-hill-r...
1,1,"NBA, WNBA Ramp Up Efforts To Get Out The Vote ...",off-season news,http://global.nba.com/news/nba-wnba-ramp-up-ef...
2,2,"Pat Riley Discusses Heat Future, Plans For Kee...",off-season news,http://global.nba.com/news/pat-riley-discusses...
3,3,Report: Amar’e Stoudemire Joins Steve Nash’s C...,off-season news,http://global.nba.com/news/report-amare-stoude...
4,4,ESPN To Host Virtual 2020 NBA Draft Presented ...,off-season news,http://global.nba.com/news/espn-to-host-virtua...
...,...,...,...,...
5995,5995,"Antetokounmpo Scores 34, Bucks Top Spurs 135-129",2018-19,http://global.nba.com/news/antetokounmpo-score...
5996,5996,Mavericks Pull Away In 4th For 113-104 Win Ove...,2018-19,http://global.nba.com/news/mavericks-pull-away...
5997,5997,"Murray Leads Nuggets Past Thunder, 105-98",2018-19,http://global.nba.com/news/murray-leads-nugget...
5998,5998,"Towns’ 35 Points, 22 Boards Lead Wolves Over B...",2018-19,http://global.nba.com/news/towns-35-points-22-...


,Unnamed: 0,title,time,url
0,0,Warriors’ Stephen Curry To Begin Practicing Ne...,2018-19,http://global.nba.com/news/warriors-stephen-cu...
1,1,Pelicans’ Anthony Davis (Hip Strain) Out vs. W...,2018-19,http://global.nba.com/news/pelicans-anthony-da...
2,2,Rockets To Rest Chris Paul Against Cavaliers,2018-19,http://global.nba.com/news/rockets-to-rest-chr...
3,3,Warriors Looking To Stay On Track As They Host...,2018-19,http://global.nba.com/news/warriors-looking-to...
4,4,Bucks End Homestand Against Spurs,2018-19,http://global.nba.com/news/bucks-end-homestand...
...,...,...,...,...
5995,5995,Returning champion Klay Thompson headlines JBL...,2016-17,http://global.nba.com/news/returning-champion-...
5996,5996,"Aaron Gordon, DeAndre Jordan Headline 2017 Ver...",2016-17,http://global.nba.com/news/aaron-gordon-deandr...
5997,5997,Keys to the Game: Bulls vs. Warriors (03.02.17),2016-17,http://global.nba.com/news/keys-to-the-game-bu...
5998,5998,Hornets Acquire Center Miles Plumlee From Buck...,2016-17,http://global.nba.com/news/hornets-acquire-cen...


,Unnamed: 0,title,time,url
0,0,Isaiah Thomas Named Eastern Conference Player ...,2016-17,http://global.nba.com/news/isaiah-thomas-named...
1,1,Muhammad Anchoring Wolves Second Unit,2016-17,http://global.nba.com/news/muhammad-anchoring-...
2,2,Schedule Breakdown: Clippers in for challengin...,2016-17,http://global.nba.com/news/schedule-breakdown-...
3,3,Magic Johnson to Advise Lakers Team Ownership,2016-17,http://global.nba.com/news/magic-johnson-to-ad...
4,4,Feb. 2 Shootaround — Knicks Still In Pursuit O...,2016-17,http://global.nba.com/news/feb-2-shootaround-k...
...,...,...,...,...
4391,4391,2015-16 Season Preview Extravaganza | How Will...,off-season news,http://global.nba.com/news/2015-16-season-prev...
4392,4392,All NBA League Pass First Team: John Wall,off-season news,http://global.nba.com/news/all-nba-league-pass...
4393,4393,All NBA League Pass First Team: Damian Lillard,off-season news,http://global.nba.com/news/all-nba-league-pass...
4394,4394,All NBA League Pass First Team: Giannis Anteto...,off-season news,http://global.nba.com/news/all-nba-league-pass...
